In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

True 2.1.1+cu118


In [4]:
from src.api import Trainer  
app = Trainer.initialize(stage = 0 , resume = 0 , checkname= 1)
app.go()

/home/mengkjin/Workspace/learndl/src/nn_model/util/logger.py:22: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/mengkjin/Workspace/learndl/logs/logs/rnn_log.log' mode='a' encoding='utf-8'>
  log.removeHandler(log.handlers[-1])
24-07-26 07:25:24|MOD:display     |: Model Specifics:


24-07-26 07:25:24|MOD:display     |: Start Process [Data] at Fri Jul 26 07:25:24 2024!


risk_att_gru nn
--Process Queue : Data + Fit + Test
--Start Training New!
--Model_name is set to risk_att_gru_day.3!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True,kwargs={}) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2,kwargs={}) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20,kwargs={}) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05,kwargs={}) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1],kwargs={}) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4,kwargs={}) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2,kwargs={}) , display batch progress bar
Callback : StatusDisplay(verbosity=2,kwargs={}) , display epoch and event information
Callback : GroupReturnAnalysis(gr

24-07-26 07:25:29|MOD:display     |: Finish Process [Data], Cost 4.8 Secs
24-07-26 07:25:29|MOD:display     |: Start Process [Fit] at Fri Jul 26 07:25:29 2024!
/home/mengkjin/Workspace/learndl/src/nn_model/classes/mod.py:345: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.logger.warn(f'First Iterance: ({self.status.model_date} , {self.status.model_num})')
24-07-26 07:25:29|MOD:mod         |: First Iterance: (20170103 , 0)


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
Pre-Norming method of [style] : {'divlast': False, 'histnorm': False}
Pre-Norming method of [indus] : {'divlast': False, 'histnorm': False}


...

,group,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
num,type,,,,,,,,,,,,,,,,,,,,
0,best,-2.081%,-0.861%,-0.497%,-0.320%,-0.206%,-0.158%,-0.040%,-0.024%,0.058%,0.114%,0.184%,0.242%,0.263%,0.319%,0.349%,0.445%,0.451%,0.551%,0.524%,0.749%
1,best,-2.022%,-0.761%,-0.502%,-0.287%,-0.244%,-0.122%,0.005%,-0.016%,0.037%,0.104%,0.096%,0.168%,0.263%,0.324%,0.390%,0.400%,0.423%,0.511%,0.643%,0.650%


Grouped Return Results are saved to /home/mengkjin/Workspace/learndl/model/risk_att_gru_day.3/detailed_analysis/group.xlsx


,factor_name,benchmark,start,end,pf,bm,excess,annualized,mdd,te,ir,calmar,turnover,mdd_period
1,best,csi300,20170104,20240718,100.92%,24.21%,76.71%,6.46%,5.89%,3.84%,1.683,1.096,147.095,20201117-20211210
2,best,csi500,20170104,20240718,154.15%,-15.40%,169.55%,15.49%,14.86%,5.94%,2.608,1.043,147.263,20210317-20210915
0,best,csi1000,20170104,20240718,131.51%,-40.37%,171.88%,19.20%,14.62%,5.99%,3.204,1.313,147.337,20210429-20210915


24-07-26 11:25:16|MOD:time        |: Main Process Finished! Cost 3 Hours 59 Minutes 51.7 Seconds


Analytic datas are saved to /home/mengkjin/Workspace/learndl/model/risk_att_gru_day.3/detailed_analysis/perf.xlsx
Analytic plots are saved to /home/mengkjin/Workspace/learndl/model/risk_att_gru_day.3/detailed_analysis/plot.pdf


In [1]:
from src.api import HiddenExtractor
extractor = HiddenExtractor('resnet_gru_30m' , model_nums=[1] , model_types=['best'])
extractor.extract_hidden('update' , deploy = True)

Use device name: NVIDIA GeForce RTX 4090
try using /home/mengkjin/Workspace/learndl/data/DataSet/30m.20240703.pt , success!
Load  2 DataBlocks...... finished! Cost 1.08 secs
Align 2 DataBlocks...... finished! Cost 0.76 secs
Pre-Norming method of [30m] : {'divlast': False, 'histnorm': True}


In [1]:
from src.api import DataAPI , Trainer
DataAPI.reconstruct_train_data()
Trainer.update_models()

Use device name: NVIDIA GeForce RTX 4090
predict is False , Data Processing start!
6 datas :['y', 'day', '30m', 'style', 'indus', 'week']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 19.78 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 18.38 secs
 --> labels blocks merging (2)...... finished! Cost 3.39 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 67.56 secs
y blocks loading finished! Cost 120.07 secs
y blocks process...... finished! Cost 46.25 secs
y blocks masking...... finished! Cost 0.76 secs
y blocks saving ...... finished! Cost 3.51 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 170.84 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 29.59 secs
day blocks loading finished! Cost 29.62 secs
day blocks process...... finished! Cost 4.0

24-07-20 01:02:48|MOD:display     |: Model Specifics:
24-07-20 01:02:48|MOD:display     |: Start Process [Data] at Sat Jul 20 01:02:48 2024!


... finished! Cost 1.81 secs
week blocks norming...... finished! Cost 0.00 secs
week finished! Cost 5.83 Seconds
--------------------------------------------------------------------------------
Data Processing Finished! Cost 28.90 Seconds
--------------------------------------------------------------------------------
--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gru_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True,kwargs={}) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2,kwargs={}) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20,kwargs={}) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05,kwargs={}) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1],kwargs={}) , retrain with new lr if fitting stopped

24-07-20 01:02:50|MOD:display     |: Finish Process [Data], Cost 2.5 Secs
24-07-20 01:02:50|MOD:display     |: Start Process [Fit] at Sat Jul 20 01:02:50 2024!
24-07-20 01:02:50|MOD:mod         |: First Iterance: (20170103 , 0)
24-07-20 01:02:50|MOD:time        |: Main Process Finished! Cost 2.5 Seconds


Load  1 DataBlocks...... finished! Cost 0.02 secs
Align 1 DataBlocks...... finished! Cost 0.05 secs


KeyError: 'data.hidden'

In [ ]:
from src.api import Trainer
Trainer.update_models()

Use device name: NVIDIA GeForce RTX 4090


24-06-24 01:10:43|MOD:display     |: Model Specifics:
24-06-24 01:10:43|MOD:display     |: Start Process [Data] at Mon Jun 24 01:10:43 2024!


--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gru_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=5,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gruRES_day',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_types': ['best', 'swalast', '

24-06-24 01:10:51|MOD:display     |: Finish Process [Data], Cost 7.9 Secs
24-06-24 01:10:51|MOD:display     |: Start Process [Fit] at Mon Jun 24 01:10:51 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


...

FirstBite Ep#  0 : loss  1.00382, train-0.00516, valid-0.03649, best-0.0365, lr1.3e-03
FirstBite Ep#  5 : loss  0.86311, train 0.13764, valid 0.09228, best 0.0923, lr2.5e-03
FirstBite Ep# 10 : loss  0.84547, train 0.15544, valid 0.09630, best 0.0963, lr1.9e-03
FirstBite Ep# 15 : loss  0.83038, train 0.17066, valid 0.08826, best 0.0963, lr1.0e-07
FirstBite Ep# 20 : loss  0.82732, train 0.17363, valid 0.08575, best 0.0963, lr9.4e-04
FirstBite Ep# 25 : loss  0.81729, train 0.18370, valid 0.08407, best 0.0963, lr3.1e-04
FirstBite Ep# 30 : loss  0.81421, train 0.18647, valid 0.08262, best 0.0963, lr1.6e-04
24-06-24 01:27:46|MOD:display     |: gruRTN_day #1 @20231201|FirstBite Ep# 31 EarlyStop|Train 0.1865 Valid 0.0826 BestVal 0.0826|Cost  1.9Min,  3.6Sec/Ep
FirstBite Ep#  0 : loss  1.00293, train-0.00341, valid-0.00344, best-0.0034, lr1.3e-03
FirstBite Ep#  5 : loss  0.87709, train 0.12551, valid 0.08796, best 0.1129, lr2.5e-03
FirstBite Ep# 10 : loss  0.86180, train 0.13731, valid 0.10436,

--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gruRES_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=5,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gruRES_day',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_types': ['best', 'swalast'

24-06-24 01:36:34|MOD:display     |: Finish Process [Data], Cost 2.0 Secs
24-06-24 01:36:34|MOD:display     |: Start Process [Fit] at Mon Jun 24 01:36:34 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


FirstBite Ep#  0 : loss  1.01252, train-0.01521, valid-0.03742, best-0.0374, lr1.3e-03
FirstBite Ep#  5 : loss  0.91026, train 0.08135, valid 0.03185, best 0.0599, lr2.5e-03
FirstBite Ep# 10 : loss  0.89291, train 0.09436, valid 0.04522, best 0.0599, lr1.9e-03
FirstBite Ep# 15 : loss  0.87755, train 0.10648, valid 0.03959, best 0.0599, lr1.0e-07
FirstBite Ep# 20 : loss  0.87426, train 0.10831, valid 0.03543, best 0.0599, lr9.4e-04
FirstBite Ep# 21 : loss  0.87082, train 0.11088, valid 0.03980, best 0.0599, lr6.3e-04, Next attempt goes!
Retrain#1 Ep#  0 : loss  0.99637, train 0.00462, valid 0.03106, best 0.0311, lr1.3e-04
Retrain#1 Ep#  5 : loss  0.94196, train 0.05698, valid 0.03578, best 0.0560, lr2.5e-04
Retrain#1 Ep# 10 : loss  0.92553, train 0.06750, valid 0.03080, best 0.0560, lr1.9e-04
Retrain#1 Ep# 15 : loss  0.91653, train 0.07499, valid 0.03537, best 0.0560, lr1.0e-07
Retrain#1 Ep# 20 : loss  0.91509, train 0.07561, valid 0.04185, best 0.0560, lr9.4e-05
Retrain#1 Ep# 21 : loss